# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [187]:
import math
import os, requests
import random
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import numpy as np
import math

load_dotenv()
API_KEY = os.environ.get('key')
REGION = 'na1' 
root = 'https://americas.api.riotgames.com'
winRateDict = {}
pickRateDict = {}
top5WR = {}
champList = ['aphelios', 'ashe', 'caitlyn', 'corki', 'draven', 'ezreal', 'jhin', 'jinx', 'kaisa', 'kalista', 'kogmaw', 'lucian', 'missfortune', 'nilah', 'samira', 'sivir', 'smolder', 'tristana', 'twitch', 'varus', 'vayne', 'xayah', 'zeri']
champListLength = len(champList)
headers = {
    'User-Agent': 'Mozilla/5.0 (compatible; YourAppName/1.0)'
}

In [188]:
def updateWinRates(winRateDict, headers):
    for champ in champList:
        opggURL = f'https://www.op.gg/champions/{champ}/build'
        response = requests.get(opggURL, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            winRateEm = soup.find('em', string='Win rate')
            winRateB = winRateEm.find_next('b', class_='text-[12px]')
            if winRateB:
                raw_text = winRateB.get_text(separator=" ", strip=True)
                winRate = raw_text.replace('"', '').strip()
                winRateDict.update({champ: float(winRate.strip(" %"))})
        else:
            print(f"Failed to retrieve data: {response.status_code}")
            
def updatePickRates(pickRateDict, headers):
    for champ in champList:
        opggURL = f'https://www.op.gg/champions/{champ}/build'
        response = requests.get(opggURL, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            pickRateEm = soup.find('em', string='Pick rate')
            pickRateB = pickRateEm.find_next('b', class_='text-[12px]')
            if pickRateB:
                raw_text = pickRateB.get_text(separator=" ", strip=True)
                pickRate = raw_text.replace('"', '').strip()
                pickRateDict.update({champ: float(pickRate.strip(" %"))})
        else:
            print(f"Failed to retrieve data: {response.status_code}")

def getTop5(top5, winRateDict):
    for key, value in winRateDict.items():
        currWR = float(winRateDict.get(key))
        if len(top5) < 5:
            top5[key] = currWR
        else:
            smallest_key = min(top5, key=top5.get)
            if currWR > top5[smallest_key]:
                top5.pop(smallest_key)
                top5[key] = currWR

def getRandomTop5WR(top5WR):
    keyList = list(top5WR.keys())
    randomKey = random.choice(keyList)
    print(f'Your random champion is {randomKey.capitalize()}!\nWin rate: {top5WR[randomKey]}')
    
def pickRandomChamp(champList):
    return champList[random.randint(0, champListLength - 1)]

def addZScores(dictionary):
    zScoreDict = {}
    prSTD = np.std(list(dictionary.values()))
    prMean = np.mean(list(dictionary.values()))
    for key, value in dictionary.items():
        floatVal = float(value)
        currentKey = key
        zScore = float((floatVal - prMean) / prSTD)
        zScoreDict[key] = zScore
    return zScoreDict

def joinZPicKRate(winRateDict, zscoreDict):
    result = {}
    for key, value in winRateDict.items():
        result.update({key: 1.5 * value + 2 * zscoreDict[key]})
    return result

In [189]:
updateWinRates(winRateDict, headers)

In [190]:
updatePickRates(pickRateDict, headers)

In [191]:
getTop5(top5WR, winRateDict)

In [192]:
getRandomTop5WR(top5WR)

Your random champion is Tristana!
Win rate: 51.82


In [193]:
zScorePRDict = addZScores(pickRateDict)
zScoreWRDict = addZScores(winRateDict)

In [195]:
truePowerLevelDict = joinZPicKRate(zScoreWRDict, zScorePRDict)

top5PL = dict(sorted(truePowerLevelDict.items(), key=lambda item: item[1], reverse=True)[:5])
truePowerLevelDict = dict(sorted(truePowerLevelDict.items(), key=lambda item: item[1], reverse=True))
sortedWR = dict(sorted(winRateDict.items(), key=lambda item: item[1], reverse=True))
sortedPR = dict(sorted(pickRateDict.items(), key=lambda item: item[1], reverse=True))

print(f'Win Rates: {sortedWR}\n')
print(f'Pick Rates: {sortedPR}\n')
print(f'Power Level: {truePowerLevelDict}\n')
print(f'Top 5: {top5PL}\n')

Win Rates: {'nilah': 53.74, 'kogmaw': 53.41, 'vayne': 52.27, 'jinx': 52.01, 'tristana': 51.82, 'twitch': 51.36, 'missfortune': 51.31, 'lucian': 51.22, 'sivir': 51.06, 'samira': 50.79, 'jhin': 50.69, 'xayah': 50.62, 'draven': 50.54, 'zeri': 50.46, 'ashe': 50.44, 'corki': 50.43, 'aphelios': 50.19, 'smolder': 50.16, 'kaisa': 49.74, 'caitlyn': 49.69, 'varus': 49.68, 'kalista': 48.66, 'ezreal': 47.82}

Pick Rates: {'kaisa': 21.37, 'jhin': 19.12, 'ezreal': 19.06, 'jinx': 17.12, 'caitlyn': 12.52, 'lucian': 11.19, 'tristana': 10.83, 'ashe': 9.09, 'varus': 7.4, 'missfortune': 6.97, 'vayne': 6.11, 'aphelios': 5.6, 'smolder': 5.46, 'xayah': 5.07, 'draven': 4.78, 'twitch': 4.29, 'sivir': 4.23, 'corki': 3.52, 'zeri': 3.49, 'samira': 3.47, 'nilah': 2.01, 'kogmaw': 2.0, 'kalista': 1.87}

Power Level: {'jinx': 4.493207328775545, 'jhin': 3.664140828149875, 'kaisa': 3.3455088441653063, 'tristana': 2.117604512920501, 'lucian': 1.5524133254077368, 'nilah': 1.296028125819241, 'vayne': 1.0151106535834602, '